# Kernel density estimate with edge correction

We follow:

Kelsall, Diggle, "Non-parametric estimation of spatial variation in relative risk", Statistics in Medicine 14 (1995) 2335--2342

The idea is bias the density estimation near the boundary of a study region to reduce "edge effects".

We assume that we are using a fixed bandwidth, Gaussian KDE, of the form

$$ f(x) = \frac{1}{n} \sum_{i=1}^n \frac{1}{|S|^{1/2}} \frac{1}{h^2} K\big(h^{-2} (x-x_i)^T S^{-1} (x-x_i)\big)
\qquad (x\in\mathbb R^2), $$
where $K(t) = (2\pi)^{-1} e^{-t/2}$.  This is slightly more general than Kelsall and Diggle, as we allow a "covariance matrix" $S$.  The "edge corrected" estimate is
$$ \frac{f(x)}{q(x)} \quad\text{where}\quad q(x) = \int_A \frac{1}{|S|^{1/2}} \frac{1}{h^2} K\big(h^{-2} (x-y)^T S^{-1} (x-y)\big)
dy, $$
where $A$ is our study region.

Let us first [change variables](https://en.wikipedia.org/wiki/Integration_by_substitution#Substitution_for_multiple_variables)
\begin{align*} q(x) &=
\int_{S^{-1/2}A} \frac{1}{h^2} K\big(h^{-2} (x-S^{1/2}z)^T S^{-1} (x-S^{1/2}z)\big) dz, \\
&= \int_{S^{-1/2}A} \frac{1}{h^2} K\big(h^{-2} (S^{-1/2}x-z)^T (S^{-1/2}x-z)\big) dz \\
&= \int_{S^{-1/2}A} \frac{1}{2\pi h^2} e^{-\|(S^{-1/2}x-z)/h\|^2/2} dz.
\end{align*}

We can now follow Kelsall and Diggle.  Choose an integer $m$ and $r_1 < r_2 < \cdots < r_m$ with 
$$ \int_{0}^{r_i} \frac{1}{2\pi h^2} e^{-(r/h)^2/2} 2\pi r \ dr = \frac{i-1/2}{m}. $$
This is, we work in polar coordinates.  Notice that this integral can be evaluated exactly,
$$ \int_{0}^s \frac{1}{h^2} e^{-(r/h)^2/2} r \ dr = \int_{0}^s - \frac{d}{dr} e^{-(r/h)^2/2} \ dr
= 1 - e^{-(s/h)^2/2}, $$
and so
$$ r_i^2 = -2h^2 \log\Big( \frac{m - i + 1/2}{m} \Big). $$

Then we consider each circle of radius $r_i$ centred at $S^{-1/2}x$, and consider $k$ evenly spaced points on this circle.  Let $p(S^{-1/2}x, r_i)$ be the number of these $k$ points inside $S^{-1/2}A$.  Then,
$$ q(x) \approx \frac{1}{m} \sum_{i=1}^m p(S^{-1/2}x, r_i). $$
Recommended values are $m=10, k=100$.